<a href="https://colab.research.google.com/github/ambarja/INLA-MODELS/blob/main/inla_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" style="padding-right:10px;" src="https://avatars.githubusercontent.com/u/46831228?s=200&v=4" width=10% ><img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon.ico" width=10% >
<p><b><h2 align ="center">Malaria Hydrobasins project</h2></b></p>
<p>Este proyecto tiene como objetivo analizar de forma espacial y temporal las tasas de malaria en la región de Loreto a nivel distrital y microcuenas empleando modelos bayesianos como INLA. <img src = 'https://img.shields.io/github/license/Naereen/StrapDown.js.svg' href='https://github.com/Naereen/StrapDown.js/blob/master/LICENSE'> </p>


In [ ]:
#@title **Install packages main. It takes aprox. 10 min**
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
system('sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev')
install.packages('tidyverse')
install.packages("INLA",repos=c(getOption("repos"),INLA="https://inla.r-inla-download.org/R/testing"), dep=TRUE)
install.packages('spdep')
install.packages('Metrics')
install.packages('spatialreg')

In [11]:
#@title **functions - inla_utils**
inla_function <- function(data, eq) {
  result <- inla(eq,
    family = "nbinomial",
    verbose = F,
    data = data,
    E = nrohab,
    control.compute = list(dic = TRUE, waic = TRUE, cpo = TRUE),
    control.predictor = list(compute = TRUE, link = 1)
  )
}


metricas_error <- function(data, model){
  data$pred <- model$summary.fitted.values$`0.5quant`
  met_rmse <- rmse(actual = data$fal , predicted = data$pred)
  met_rse  <- rse(actual = data$fal , predicted = data$pred)
  met_msle <- msle(actual = data$fal , predicted = data$pred)
  met_mae <- mae(actual = data$fal , predicted = data$pred)
  met_waic <- model$waic[1]
  met_dic <- model$dic[1]
      
  met_tabla <- tibble(rmse = met_rmse, rse = met_rse,
                      msle = met_msle, mae = met_mae,
                      waic = met_waic, dic = met_dic )
  return(met_tabla)
}


In [12]:
library(INLA)
library(tidyverse)
library(purrr)
library(spdep)
library(Metrics)

##1.Reading to dataset and spatial data

In [13]:
spatial <- read_rds("/content/sp_district.rds")
dataset <- read_rds("/content/alldataset_district.rds") %>% mutate(month = as.integer(month))

##2.Calculate the neighborhood matrix

In [14]:
nb <- poly2nb(spatial)
g <- inla.read.graph(nb)

##3.Models spatial, temporal and spatial-*temporal*

In [22]:
types <- c(
  # esco
  formula = fal ~ esco + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # etp
  formula = fal ~ etp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # evi
  formula = fal ~ evi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # hsoil
  formula = fal ~ hsoil + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # ndvi
  formula = fal ~ ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # pp
  formula = fal ~ pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # savi
  formula = fal ~ savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # temp
  formula = fal ~ temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp
  formula = fal ~ esco + etp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-evi
  formula = fal ~ esco + evi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-hsoil
  formula = fal ~ esco + hsoil + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-ndvi
  formula = fal ~ esco + ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),  
  # esco-pp
  formula = fal ~ esco + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # esco-savi
  formula = fal ~ esco + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # esco-temp
  formula = fal ~ esco + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # etp-evi
  formula = fal ~ etp + evi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # etp-hsoil
  formula = fal ~ etp + hsoil + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # etp-ndvi
  formula = fal ~ etp + ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # etp-pp
  formula = fal ~ etp + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # etp-savi
  formula = fal ~ etp + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"), 
  # etp-temp
  formula = fal ~ etp + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # evi-hsoil
  formula = fal ~ evi + hsoil + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # evi-ndvi
  formula = fal ~ evi + ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # evi-pp
  formula = fal ~ evi + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # evi-savi
  formula = fal ~ evi + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # evi-temp
  formula = fal ~ evi + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # hsoil-ndvi
  formula = fal ~ hsoil + ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # hsoil-pp
  formula = fal ~ hsoil + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # hsoil-savi
  formula = fal ~ hsoil + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # hsoil-temp
  formula = fal ~ hsoil + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # ndvi-pp
  formula = fal ~ ndvi + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # ndvi-savi
  formula = fal ~ ndvi + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # ndvi-temp
  formula = fal ~ ndvi + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # pp-savi
  formula = fal ~ pp + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # pp-temp
  formula = fal ~ pp + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # savi-temp
  formula = fal ~ savi + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi
  formula = fal ~ esco + etp + evi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-hsoil
  formula = fal ~ esco + etp + hsoil + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-ndvi
  formula = fal ~ esco + etp + ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-pp
  formula = fal ~ esco + etp + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-savi
  formula = fal ~ esco + etp + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-temp
  formula = fal ~ esco + etp + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil
  formula = fal ~ esco + etp + evi + hsoil + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-ndvi
  formula = fal ~ esco + etp + evi + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-pp
  formula = fal ~ esco + etp + evi + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-savi
  formula = fal ~ esco + etp + evi + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-temp
  formula = fal ~ esco + etp + evi + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi
  formula = fal ~ esco + etp + evi + hsoil + ndvi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-pp
  formula = fal ~ esco + etp + evi + hsoil + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-savi
  formula = fal ~ esco + etp + evi + hsoil + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-temp
  formula = fal ~ esco + etp + evi + hsoil + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi-pp
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi-savi
  formula = fal ~ esco + etp + evi + hsoil + ndvi + savi + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi-temp
  formula = fal ~ esco + etp + evi + hsoil + ndvi + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi-pp-savi
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi +f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi-pp-temp
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2"),
  # esco-etp-evi-hsoil-ndvi-pp-savi-temp
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp + f(new_id, model = "bym", graph = g) + f(year, model = "iid") + f(month, model = "rw2")
)
 

##4.Names of models

In [ ]:
model_name <- c("esco","etp","evi","hsoil","ndvi","pp","savi","temp",
                "esco-etp","esco-evi","esco-hsoil","esco-ndvi","esco-pp","esco-savi","esco-temp","etp-evi",
                "etp-hsoil","etp-ndvi","etp-pp","etp-savi","etp-temp","evi-hsoil","evi-ndvi","evi-pp","evi-savi",
                "evi-temp","hsoil-ndvi","hsoil-pp","hsoil-savi","hsoil-temp",
                "ndvi-pp","ndvi-savi","ndvi-temp","pp-savi","pp-temp","savi-temp",
                "esco-etp-evi","esco-etp-hsoil","esco-etp-ndvi","esco-etp-pp","esco-etp-savi","esco-etp-temp",
                "esco-etp-evi-hsoil","esco-etp-evi-ndvi","esco-etp-evi-pp","esco-etp-evi-savi","esco-etp-evi-temp",
                "esco-etp-evi-hsoil-ndvi","esco-etp-evi-hsoil-pp","esco-etp-evi-hsoil-savi","esco-etp-evi-hsoil-temp",
                "esco-etp-evi-hsoil-ndvi-pp","esco-etp-evi-hsoil-ndvi-savi","esco-etp-evi-hsoil-ndvi-temp",
                "esco-etp-evi-hsoil-ndvi-pp-savi","esco-etp-evi-hsoil-ndvi-pp-temp","full")
 
names(types) <- model_name

##5.Run models

In [ ]:
model_types <- types %>% purrr::map(~ inla_function(data = dataset, eq = .))

##6.Metrics

In [ ]:
model_metricas <- model_types %>% 
  purrr::map(~ metricas_error(data = dataset,model = .)) %>%
  map_df(.f = data.frame,.id = 'names')

In [ ]:
model_metricas

##7.Export models and metrics

In [ ]:
#saveRDS(model_types,'/content/resultados_distritos_inla.rds')
#saveRDS(model_metricas,'/content/metricas_distritos_inla.rds')